In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import h3
import pandas as pd
import geopandas as gpd
import numpy as np
from src.settings import *
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from src.tools.osmnx_utils import get_place_dir_name
from src.tools.h3_utils import get_resolution_buffered_suffix, get_edges_with_features_filename
from pathlib import Path
import plotly.express as px
import matplotlib.pyplot as plt
import contextily as ctx
from keplergl import KeplerGl
from src.tools.aggregation import aggregate_hex
import json5 as json
tqdm.pandas()

In [20]:
with open(RAW_DATA_DIR / "featureset_transformation_default.jsonc", "r") as f:
    FEATURESET = json.load(f)

In [30]:
cities = pd.read_csv(RAW_DATA_DIR / "cities.csv")
# cities = cities[(cities.country == "Poland") & (cities.kacper)]
cities

,city,country,kacper,szymon,piotr,kamil,regions
0,Tirana,Albania,False,True,False,False,NaN
1,Yerevan,Armenia,False,True,False,False,NaN
2,Sydney,Australia,True,False,False,False,NaN
3,Vienna,Austria,False,True,True,True,NaN
4,Baku,Azerbaijan,False,True,False,False,NaN
...,...,...,...,...,...,...,...
112,Los Angeles,United States,True,True,False,False,NaN
113,New York City,United States,True,True,False,False,NaN
114,Philadelphia,United States,False,True,False,False,NaN
115,San Francisco,United States,True,True,False,False,NaN


In [31]:
resolution = 9
buffered = True
network_type = "drive"
intersection_based = False

pbar = tqdm(cities.itertuples(), total=cities.shape[0])
hexagons = []
cities_agg = []
edges = []

features_top_level = list(FEATURESET.keys())
df_feature_counts = pd.DataFrame(data={"feature": features_top_level, "count": 0 }).set_index("feature")
n = 0
for row in pbar:
    place_name = f"{row.city},{row.country}"
    place_dir_name = get_place_dir_name(place_name)
    place_dir_path = GENERATED_DATA_DIR / place_dir_name
    gpkg_path = place_dir_path / f"graph_{network_type}.gpkg"
    pbar.set_description(place_name)

    try:

        edges_city = gpd.read_feather(place_dir_path / get_edges_with_features_filename(network_type, resolution, buffered, intersection_based))
        n += len(edges_city)

        edges_city["city"] = row.city
        edges_city["country"] = row.country

        for feature in features_top_level:
            columns = [x for x in edges_city.columns if feature in x]
            count = edges_city[columns].max(axis=1).sum()
            df_feature_counts.loc[feature] += count

        city_agg = edges_city.drop(columns=["u", "v", "key", "id"]).groupby(["country", "city"]).sum()
        cities_agg.append(city_agg)
        # edges.append(edges_city)
    except Exception as e:
        print("\nFailed", place_name, e)
    

df_cities_agg = pd.concat(cities_agg)
df_agg = df_cities_agg.sum()
# edges = pd.concat(edges).reset_index()

del edges_city



Washington D.C.,United States: 100%|██████████| 117/117 [01:41<00:00,  1.15it/s]


In [32]:
edges

[]

In [33]:
df_cities_agg

oneway_False  oneway_True  lanes_1  lanes_2  \
country       city                                                           
Albania       Tirana                   8108         2662     2966     2716   
Armenia       Yerevan                 15983         3622      315     1950   
Australia     Sydney                 163393        52854     8349    26368   
Austria       Vienna                  17516        17407     3115     5545   
Azerbaijan    Baku                    93887        10176      311     2613   
...                                     ...          ...      ...      ...   
United States Los Angeles            102331        15283     2425    21580   
              New York City           66984        65786     5960    10585   
              Philadelphia            28246        29638     3324     9026   
              San Francisco           15284         7074      685     4970   
              Washington D.C.         15402         8033     2077     6049   

                               lanes_3  lanes_4  lanes_5  lanes_6  lanes_7  \
country       city                                                           
Albania       Tirana               289      159        6        0        0   
Armenia       Yerevan             1504     1298       48       33        0   
Australia     Sydney              7684     4521      673      201       19   
Austria       Vienna              2302     1045      261       36        2   
Azerbaijan    Baku                1401      459       36       52        4   
...                                ...      ...      ...      ...      ...   
United States Los Angeles         4958    10745     6783     2824     1709   
              New York City       6475     4179      991      308      140   
              Philadelphia        2533     1321      578      182       75   
              San Francisco       2214     1269      307       20        1   
              Washington D.C.     2087     1983      230      348       40   

                               lanes_8  ...  width_27.0  width_27.5  \
country       city                      ...                           
Albania       Tirana                 0  ...           0           0   
Armenia       Yerevan                0  ...           0           0   
Australia     Sydney                 2  ...           0           0   
Austria       Vienna                 0  ...           0           0   
Azerbaijan    Baku                   6  ...           0           0   
...                                ...  ...         ...         ...   
United States Los Angeles          193  ...           0           0   
              New York City          1  ...           0           0   
              Philadelphia           1  ...           0           0   
              San Francisco          4  ...           0           0   
              Washington D.C.       43  ...           0           0   

                               width_28.0  width_28.5  width_29.0  width_29.5  \
country       city                                                              
Albania       Tirana                    0           0           0           0   
Armenia       Yerevan                   0           0           0           0   
Australia     Sydney                    0           0           0           0   
Austria       Vienna                    0           0           0           0   
Azerbaijan    Baku                      0           0           0           0   
...                                   ...         ...         ...         ...   
United States Los Angeles               0           0           0           0   
              New York City             0           0           0           0   
              Philadelphia              0           0           0           0   
              San Francisco             0           0           0           0   
              Washington D.C.           0           0           0           0   

                       

In [34]:
df_agg

oneway_False                  2613321
oneway_True                   1340843
lanes_1                        253763
lanes_2                        628625
lanes_3                        200032
                               ...   
width_29.5                          0
width_30.0                         39
tunnel_building_passage          3182
tunnel_yes                      22461
tunnel_avalanche_protector          0
Length: 143, dtype: int64

In [35]:
n

3954147

In [36]:
df_feature_counts

,count
feature,
oneway,3954147
lanes,1112892
highway,3951703
maxspeed,1427048
bridge,102072
access,32140
junction,81012
width,21314
tunnel,25545


In [37]:
fig = px.bar(df_feature_counts * 100 / n, width=1300, title=f"Feature occurence out of total: {n}")
fig.update_layout(
    # xaxis = dict(
    #     tickmode = 'linear',
    # ),
    showlegend = False   
)
fig.show()

In [38]:
for feature in features_top_level:
    columns = list(filter(lambda x: x in df_agg, [f"{feature}_{value}" for value in FEATURESET[feature]]))
    fig = px.bar(df_agg[columns] * 100 / df_agg[columns].sum(), width=1300, title=feature, log_y=True)
    fig.update_layout(
        xaxis = dict(
            tickmode = 'linear',
        ),
        showlegend=False
    )
    fig.show()